### Compare Listing 
<li>vector uniform</li>
<li>greedy</li>
<li>e - greedy</li>
<li>Linear Reward Penalty</li>
<li>Linear Reward Inaction</li>
<li>UBC</li>
<li>BayesianUCB</li>
<li>Thompson Sampling</li>
<li>Neural Network</li>
<li>Non Stationary</li>

In [10]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Process, Array,Value
import scipy,time

In [11]:
# The wrong way
def func(val):
    for i in range(50):
        time.sleep(0.01)
        val.value += 1

v = Value('i', 0)
procs = [Process(target=func, args=(v,)) for i in range(10)]

for p in procs: p.start()
for p in procs: p.join()

print(v.value)

0
